In [152]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim
import scipy
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [2]:
f = h5py.File('../../../../Downloads/human_matrix.h5', 'r')
expression = f['data']['expression']

In [113]:
num_samples = 5000
num_genes = 500
expression_small = expression[np.sort(np.random.choice(expression.shape[0],num_samples,replace=False))]

In [114]:
expression_small = expression_small[:,np.sort(np.random.choice(expression_small.shape[1],num_genes,replace=False))]

In [115]:
expression_small.shape

(5000, 500)

In [638]:
# simulated measurements
measurements = 300
A = np.random.normal(size=(measurements,num_genes))
compressed_mes = np.dot(A,expression_small.T).T
X_train, X_test,y_train, y_test = train_test_split(compressed_mes,expression_small,test_size=0.2)
X_train_std = np.std(X_train,axis=0)
X_train_mean = np.mean(X_train,axis=0)
X_train = np.divide(X_train - X_train_mean,X_train_std)
X_test = np.divide(X_test - X_train_mean,X_train_std)

# perform standardization on labels
y_train_std = np.std(y_train,axis=0)
y_train_mean = np.mean(y_train,axis=0)
y_stand = np.divide(y_train - y_train_mean,y_train_std)

In [639]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(measurements,200)  # 6*6 from image dimension
        self.fc2 = nn.Linear(200, 300)
#         self.fc4 = nn.Linear(300, 300) #added
        self.fc3 = nn.Linear(300, num_genes)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
#         # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = F.relu(self.fc4(x)) #added
        x = F.relu(self.fc3(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net = net.double()
print(net)

Net(
  (fc1): Linear(in_features=300, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=500, bias=True)
)


In [640]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

6
torch.Size([200, 300])


In [641]:
X_train = torch.from_numpy(X_train.astype(float))
X_test = torch.from_numpy(X_test.astype(float))
y_train = torch.from_numpy(y_train.astype(float))
y_test = torch.from_numpy(y_test.astype(float))
y_stand = torch.from_numpy(y_stand.astype(float))

In [642]:
X_train.shape

torch.Size([4000, 300])

In [643]:
criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00000000000001)
optimizer = optim.SGD(net.parameters(), lr=0.0001)



In [644]:
out= net(X_train[0,:])
print(out)

tensor([0.0853, 0.0000, 0.1209, 0.0000, 0.0238, 0.0000, 0.0000, 0.0000, 0.0289,
        0.0000, 0.0317, 0.0035, 0.0000, 0.0000, 0.0000, 0.0000, 0.0691, 0.0748,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0034, 0.0094, 0.0279, 0.0663, 0.0749,
        0.0000, 0.0662, 0.0906, 0.0356, 0.1655, 0.0000, 0.1049, 0.0455, 0.1046,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1807, 0.1028,
        0.0000, 0.0716, 0.0000, 0.0263, 0.0000, 0.0311, 0.0000, 0.0166, 0.0177,
        0.0000, 0.0000, 0.0000, 0.0088, 0.0000, 0.0140, 0.0000, 0.1871, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0124, 0.0000, 0.0787, 0.1091, 0.0000, 0.1127,
        0.0886, 0.0000, 0.0536, 0.0007, 0.0000, 0.0630, 0.0459, 0.0000, 0.0822,
        0.0953, 0.0000, 0.0000, 0.0000, 0.0689, 0.0203, 0.0442, 0.0060, 0.1770,
        0.0000, 0.0321, 0.0514, 0.0838, 0.0000, 0.0000, 0.0000, 0.0782, 0.0893,
        0.0456, 0.1656, 0.0000, 0.0000, 0.1241, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0739, 0.0000, 

In [645]:
for epoch in range(100):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(X_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data
        labels = y_stand[i,:]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
#         print(inputs)
        outputs = net(inputs)
#         print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 1000 == 999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 100))
#             running_loss = 0.0
    print(running_loss)

print('Finished Training')

0
3993.6661970086984
1
3990.8121475330186
2
3988.1542405058353
3
3985.613976722891
4
3983.1474752377558
5
3980.7418489675
6
3978.3730525280944
7
3976.0271033336903
8
3973.697810457038
9
3971.3567553657335
10
3968.999427208863
11
3966.6144881410937
12
3964.17048411419
13
3961.685406517444
14
3959.137182174774
15
3956.5164903392565
16
3953.831289018844
17
3951.065835174636
18
3948.2211280080346
19
3945.2978752908593
20
3942.2911826272116
21
3939.1836294980076
22
3935.9896486350226
23
3932.7234565630747
24
3929.3767819954496
25
3925.957274490393
26
3922.464065958507
27
3918.9113685347115
28
3915.2845922220627
29
3911.5979559769194
30
3907.8788071763597
31
3904.1212234351883
32
3900.320702328775
33
3896.494790606937
34
3892.627959607259
35
3888.7417047791487
36
3884.8559519257365
37
3880.9501895121175
38
3877.0465851011995
39
3873.1639069689827
40
3869.299128536111
41
3865.4509287687
42
3861.6318732285717
43
3857.8532524590646
44
3854.076477308363
45
3850.3230823882805
46
3846.623206717093

In [646]:
X_test_hat = np.zeros((y_test.shape[0],y_test.shape[1]))
total_correlation = 0
for i, data in enumerate(X_test,0):
    out = net(data)
#     pearson_corr = scipy.stats.pearsonr(out.data.numpy(),y_test[i,:].data.numpy())
    out = np.multiply(out.data.numpy()+y_train_mean,y_train_std)
    pearson_corr = scipy.stats.pearsonr(out,y_test[i,:].data.numpy())
    if (math.isnan(pearson_corr[0])):
        total_correlation = total_correlation+0
    else:
        total_correlation = pearson_corr[0] + total_correlation
print(total_correlation/y_test.shape[0])

0.6443675352270476


In [647]:
import math
total=0.0
count = 0
for i, data in enumerate(X_train,0):
    out = net(data)
#     pearson_corr = scipy.stats.pearsonr(out.data.numpy(),y_train[i,:].data.numpy())
    out = np.multiply(out.data.numpy()+y_train_mean,y_train_std)
    pearson_corr = scipy.stats.pearsonr(out,y_train[i,:].data.numpy())
    if (math.isnan(pearson_corr[0])):
        total = total+0
    else:
        total = pearson_corr[0] + total
        count = count + 1
#     print(total)
print(total/count)

0.6465869838090701


In [567]:
y_train_std.shape

(500,)

## running neural net with sci-kit

In [296]:
import math

In [554]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(300, 300,300, 300), max_iter=1000)
mlp.fit(X_train.data.numpy(), y_train.data.numpy())

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(300, 300, 300, 300), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [555]:
out = mlp.predict(X_test.data.numpy())

In [556]:
total_correlation = 0
for i in range(0,out.shape[0]):
    pearson_corr = scipy.stats.pearsonr(out[i,:],y_test[i,:].data.numpy())
    if (math.isnan(pearson_corr[0])):
        total_correlation = total_correlation+0
    else:
        total_correlation = pearson_corr[0] + total_correlation
print(total_correlation/y_test.shape[0])

0.7154320872341129


In [557]:
out2 = mlp.predict(X_train.data.numpy())
total_correlation = 0
for i in range(0,out2.shape[0]):
    pearson_corr = scipy.stats.pearsonr(out2[i,:],y_train[i,:].data.numpy())
    if (math.isnan(pearson_corr[0])):
        total_correlation = total_correlation+0
    else:
        total_correlation = pearson_corr[0] + total_correlation
print(total_correlation/y_train.shape[0])

0.707435239646608


/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [654]:
scipy.stats.pearsonr([1.0,1.0,2.0],[3.0,3.0,2.0])

(-1.0, 0.0)